# This notebook is aimed at download HSC cutout images, with a galaxy catalog provided.

In [1]:
import os
import urllib
import urllib2
import glob
import subprocess
import slug

import numpy as np

import astropy.wcs as wcs
from astropy.nddata import Cutout2D
from astropy.io import fits
from astropy.table import Table, Column

In [2]:
path = './'
os.chdir(path)
if not os.path.isdir('Cutout'):
    os.mkdir('Cutout')

## Import catalog

In [4]:
inter_z = Table.read('s16a_massive_z_0.5_logm_11.4_dec_30_for_john.fits', format='fits')
z_0203 = inter_z[np.logical_and.reduce([inter_z['z_best']<0.3, inter_z['z_best']>0.0])]
z_0304 = inter_z[np.logical_and.reduce([inter_z['z_best']<0.4, inter_z['z_best']>0.3])]
z_0405 = inter_z[np.logical_and.reduce([inter_z['z_best']<0.5, inter_z['z_best']>0.4])]

In [5]:
#z_0203.write('z_0203.fits', format='fits')
#z_0304.write('z_0304.fits', format='fits')
#z_0405.write('z_0405.fits', format='fits')

## Download cutout for z_0203 subset

In [6]:
pixel_scale = 0.168 # arcsec / pixel

In [7]:
slug.gen_url_hsc_s18a(351.3506808951273, -0.9463258717404696, 1000, 1000, 'r', pixel_unit=True)

['https://hscdata.mtk.nao.ac.jp/das_quarry/dr2.1/cgi-bin/cutout?ra=351.350680895&dec=-0.94632587174&sw=168.0asec&sh=168.0asec&type=coadd&image=on&mask=on&variance=on&filter=HSC-R&tract=&rerun=s18a_wide']

In [8]:
url_set = []
for obj in z_0203[:5]:
    ra, dec = obj['ra'], obj['dec']
    w, h = 100, 100
    band = 'r'
    url_set.append(gen_url(ra, dec, w, h, band))
url_set = np.array(url_set).flatten().tolist()
z_0203.add_column(Column(name='url', data=url_set))

NameError: name 'gen_url' is not defined

In [9]:
# Write the urls to a file
fh = open('filename.txt', 'w')
for url in url_set[:3]:
    fh.write(url)
    fh.write('\n')
fh.close()

### Download using urllib

In [9]:
# Import HSC username and password
config = Table.read('/Users/jiaxuanli/Documents/HSC_config', format='ascii.no_header')['col1']
username = config[0]
password = config[1]

In [11]:
# Create a password manager
password_mgr = urllib2.HTTPPasswordMgrWithDefaultRealm()

# Add the username and password.
# If we knew the realm, we could use it instead of None.
top_level_url = 'https://hscdata.mtk.nao.ac.jp/das_quarry/dr2.1/'

password_mgr.add_password(None, top_level_url, username, password)

handler = urllib2.HTTPBasicAuthHandler(password_mgr)

# create "opener" (OpenerDirector instance)
opener = urllib2.build_opener(handler)

# use the opener to fetch a URL
opener.open(top_level_url)

# Install the opener.
# Now all calls to urllib.request.urlopen use our opener.
urllib2.install_opener(opener)

In [12]:
%%time
os.chdir('/Users/jiaxuanli/Research/HSC_Dragonfly_DECaLS/IntermediateZ/Cutout/')
if not os.path.isdir('z_0203'):
    os.mkdir('z_0203')

for obj in z_0203:
    
    urllib.urlretrieve(
        obj['url'],
        filename=path + 'Cutout/z_0203/' + 'HSC-cutout-num' + str(obj.index) + '&ra=' + str(round(obj['ra'], 5)) +
        '&dec=' + str(round(obj['dec'], 5)) + '&z=' + str(round(obj['z_best'], 5)) +'&band=HSC-r.fits')

Enter username for hsc-ssp at hscdata.mtk.nao.ac.jp: huangs
Enter password for huangs in hsc-ssp at hscdata.mtk.nao.ac.jp: ········
CPU times: user 1min 3s, sys: 37.4 s, total: 1min 41s
Wall time: 1h 6min 1s


## In the downloaded `fits` files, layer[1] is image, layer[2] is mask (now empty), and layer[3] is variance.